In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

### Reading the Dataset

In [3]:
df = (spark.read
          .format("csv")
          .option('header', 'true')
          .load("./datasets/US-Chronic-dataset.csv"))

In [4]:
#df.toPandas().head()

In [5]:
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Normalizer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import MultilabelClassificationEvaluator
from pyspark.ml import Pipeline

### Selecting Features and Casting it into required Dtype

In [6]:
columns  = df.columns

dataset = df.select(col(columns[3]).cast('string'),
                    col(columns[4]).cast('string'),
                    col(columns[5]).cast('string'),
                    col(columns[6]).cast('float'),
                    col(columns[7]).cast('float'),
                    col(columns[8]).cast('float'),
                    col(columns[9]).cast('string'),
                    col(columns[10]).cast('string'),
                    col(columns[11]).cast('string'),
                    col(columns[12]).cast('string'),
                    col(columns[13]).cast('string'),
                    col(columns[14]).cast('float'),
                    col(columns[15]).cast('float'),
                   )

In [7]:
# inputCols for StringIndexer
string_col = [
    'DataSource','DataValueUnit','DataValueTypeID',
    'QuestionID','LocationID','StratificationCategoryID1',
    'StratificationID1','TopicID'
]

#outputCols for StringIndexer
string_col_output = [
    'DataSourceIndex','DataValueUnitIndex','DataValueTypeIDIndex',
    'QuestionIDIndex','LocationIDIndex','StratificationCategoryID1Index',
    'StratificationID1Index','TopicIDIndex'
]

#inputCols for OneHotEncoder
string_col_encode_input = [
    'DataSourceIndex','DataValueUnitIndex','DataValueTypeIDIndex',
    'QuestionIDIndex','LocationIDIndex','StratificationCategoryID1Index',
    'StratificationID1Index'
]

#outputCols for OneHotEncoder
string_col_encoded = [
    'DataSourceVec','DataValueUnitVec','DataValueTypeIDVec',
    'QuestionIDVec','LocationIDVec','StratificationCategoryID1Vec',
    'StratificationID1Vec'
]

#inputCols for VectorAssembler
features_to_assemble = string_col_encoded + ['DataValue','LowConfidenceLimit','HighConfidenceLimit','Geo_lat','Geo_lon']

### Feature Transformation

In [8]:
indexer = StringIndexer(inputCols= string_col, outputCols=string_col_output)
encoder = OneHotEncoder(inputCols=string_col_encode_input, outputCols=string_col_encoded)
vectorAssembler = VectorAssembler(inputCols=features_to_assemble,
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)
scaler = StandardScaler(inputCol="features_norm", outputCol="features_norm_scaled")
pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer,scaler])
final_df = pipeline.fit(dataset).transform(dataset)

### Splitting the Dataset into Training data and Test data

In [9]:
(training_data, test_data) = final_df.randomSplit([0.8,0.2],seed=1234)

### Building the Model

In [10]:
rfc = RandomForestClassifier()
param_grid = (ParamGridBuilder()
              .baseOn({rfc.labelCol : 'TopicIDIndex'})
              .baseOn([rfc.predictionCol, 'pred_rfc'])
              .baseOn([rfc.featuresCol, 'features_norm'])
              .addGrid(rfc.maxDepth, [7,10,15,20,30]).build())

evaluator = MulticlassClassificationEvaluator(labelCol= 'TopicIDIndex',predictionCol= 'pred_rfc')

cv = CrossValidator(estimator=rfc, estimatorParamMaps=param_grid, evaluator=evaluator,seed=0,parallelism=2)

### Training the Model

In [11]:
cvModel = cv.fit(training_data)

In [12]:
cvModel.avgMetrics

[0.6973069875602796,
 0.8017472531901317,
 0.8840126290184793,
 0.948660583153587,
 0.9874816855658695]

### Evaluating the Model Using Different Evaluation Metrices

In [13]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "accuracy"})

0.9863996306949327

In [14]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "f1"})

0.9863540681712097

In [15]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "precisionByLabel"})

0.9994152046783625

In [16]:
evaluator.evaluate(cvModel.transform(test_data),{evaluator.metricName: "recallByLabel"})

0.9949543954977683

### Best Params

In [17]:
cvModel.bestModel.getMaxDepth()

30

### Saving the Trained Model

In [18]:
path= r'./SavedModels/RFClassifier_model'
cvModel.save(path)

### Loading the Trained Model

In [19]:
#rfc_model = CrossValidatorModel.read().load(path)

### Result Summary

In [1]:
import pandas as pd

summary_={
    'accuracy' : 0.9863996306949327,
    'f1-score' : 0.9863540681712097,
    'precision' : 0.9994152046783625,
    'recall' : 0.9949543954977683
}

summary = pd.Series(summary_)
summary

accuracy     0.986400
f1-score     0.986354
precision    0.999415
recall       0.994954
dtype: float64